<center><h1><font color="orange">Black Scholes Data Generator:</font></h1></center>

<h4>Imports:</h4>

In [90]:
import numpy as np
from math import exp, log, pi, sqrt
from scipy.stats import norm

In [91]:
class BlackScholes:
    def input_generator(self,num_samples): 
        '''
        Generate random strike/spot, time (in days), rate, volatility (sigma), and dividend data to run 
        Black-Scholes on for n sampels
        '''
        #S = np.random.uniform(0.00,100,num_samples) #Random Spot
        #KS = np.random.uniform(0.95,1.05,num_samples)  #The ratio of K/S
        #K = np.random.uniform(0.00,100,num_samples) #Random Strike -- check this
        #T = np.random.uniform(1,10*365,num_samples)   #Time In days
        #sigma = np.random.exponential(.10,num_samples)  #Random exponential sigma
        #rate = np.random.uniform(-0.2, 0.2, num_samples) # Annualized risk free rate
        #dividend = np.random.uniform(0.0, 0.2, num_samples) #dividend yield    
        KS = np.random.uniform(0.95, 1.05, num_samples)                      # K/S
        T = np.random.uniform(0, 365*10, num_samples)                      # time until expiration in days
        rate = np.random.uniform(-0.2, 0.2, num_samples)                   # Annualized risk free rate
        sigma = np.random.uniform(0.10, 2.0, num_samples)               
        dividend = np.random.uniform(0.0, 0.2, num_samples)                # Dividend yield
        X = np.asmatrix([KS,T,sigma,rate,dividend]).transpose()
        return X
    
    def getData(self,num_samples,write=False): 
        ''' 
        Get Random inputs X and prices Y
        '''
        X = self.input_generator(num_samples)
        Y = self.generate_prices(X)
        X = np.asarray(X)
        if write:
            with open('data.csv','w') as file:
                file.write('K/S,T,sigma,rate,dividend,v\n') 
                for i in range(X.shape[0]): 
                    file.write('{},{},{},{},{},{}\n'.format(X[i][0], X[i][1], X[i][2], X[i][3],X[i][4], Y[i]))
                file.close()
        return X,Y
    
    def generate_prices(self,X): 
        '''
        Vectorization to call the vanilla call price from the input
        '''
        return np.apply_along_axis(self.vanilla_call_price, axis=1, arr=X)
    

    def vanilla_call_price(self,X): 
        '''
        Calculate Vanialla Call Price
        '''

        #Get Parameters from X
        KS = X[0,0]
        T = X[0,1]
        sigma = X[0,2]
        r = X[0,3]
        d = X[0,4]

        #Black Scholes Formula Calculation
        edt = exp((-d * T) / 365)
        ert = exp((-r * T) / 365)
        sigmaTsq = sqrt(T / 365) * sigma
        #d1 = (log(edt/(K/S)) + (r + .5 * (sigma ** 2)) * T / 365) / sigmaTsq
        d1 = (log(edt/KS) + (r + .5 * (sigma ** 2)) * T / 365) / sigmaTsq
        d2 = d1 - sigmaTsq
        Nd1 = (1 + ert*(d1 / sqrt(2))) / 2
        Nd2 = (1 + ert*(d2 / sqrt(2))) / 2

        #v = edt*Nd1 - (K/S) * ert * Nd2
        v = edt*Nd1 - KS * ert * Nd2
        return v
    
    #def d_j(self,j, S, K, r, v, T):
    #    return (log(S/K) + (r + ((-1)**(j-1))*0.5*v*v)*T)/(v*(T**0.5))
    
    #def vanilla_call_price(self,X):
    #    S = X[0,0]
    #    K = X[0,1]
    #    T = X[0,2]
    #    sigma = X[0,3]
    #    r = 0
    #    return S * norm.cdf(self.d_j(1, S, K, r, sigma, T)) - \
    #        K*exp(-r*T) * norm.cdf(self.d_j(2, S, K, r, sigma, T))
    


In [92]:
b = BlackScholes()
X,Y = b.getData(2000)